# SSCN and PointNet Layer Implementation Assignment

In [1]:
#include <bits/stdc++.h>
using namespace std;

Read ModelNet10 data and Pre-process using method outlined in [Volumetric and Multi-View CNNs for Object Classification on 3D Data](https://arxiv.org/abs/1604.03265)

In [2]:
string off_file = "ModelNet10/bed/train/bed_0001.off";